In [ ]:
def index_map(type_name):
    
    text_mapping = {
      "type": "text",
      "fields": {
        "keyword": {
          "type": "keyword",
          "ignore_above": 256
        }
      }
    }
    
    n_gram = {
      "type": "text",
      "analyzer": "autocomplete",
      "search_analyzer": "autocomplete_search",
      "fields": {
        "keyword": {
          "type": "keyword",
          "ignore_above": 256
        }
      }
    }
    
    _index_mappings = {
      "settings": {
        "analysis": {
          "analyzer": {
            "autocomplete": {
              "tokenizer": "autocomplete",
              "filter": [
                "lowercase"
              ]
            },
            "autocomplete_search": {
              "tokenizer": "lowercase"
            }
          },
          "tokenizer": {
            "autocomplete": {
              "type": "edge_ngram",
              "min_gram": 4,
              "max_gram": 10,
              "token_chars": [
                "letter"
              ]
            }
          }
        }
      },
      "mappings": {
      type_name: {
        "properties": {
          "author_name": text_mapping,
          "author_join_date": {
                "type": "date",
                "format": "yyyy-MM-dd"
                },
          "author_location": text_mapping,
          "author_type": text_mapping,
          "author_url": text_mapping,
          "board": text_mapping,
          "content": n_gram,
          "floor": {
                "type": "integer"
              },
          "page": text_mapping,
          "post_date": {
                "type": "date",
                "format": "yyyy-MM-dd"
                },
          "post_time": {
                "type": "date",
                "format": "HH:mm"
              },
          "title": n_gram,
          "quotation" : text_mapping,
          "tags" : n_gram,
          "url": text_mapping
        }
       }
      }
    }
    return _index_mappings

In [ ]:
import traceback
from pymongo import MongoClient
from elasticsearch import Elasticsearch

In [ ]:
db = MongoClient('mongodb://127.0.0.1:27017')['dogs']

In [ ]:
collection = db.post

In [ ]:
es = Elasticsearch()

In [ ]:
if es.indices.exists(index='dogforum') is not True:
    es.indices.create(index='dogforum', body=index_map('posts'))

In [ ]:
cursor = collection.find({}, projection={'_id':False})

In [ ]:
docs = [x for x in cursor]
len(docs)

In [ ]:
processed = 0
for _doc in docs[:1000]:
    try:
        es.index(index='dogforum', doc_type='posts', body=_doc)
        processed += 1
    except:
        traceback.print_exc()
print('Processed: ' + str(processed))